# Intro to Cython

# Why Cython

![DevTime](whycython.png)

Outline:

* Speed up Python code
* Interact with NumPy arrays
* Release GIL and get parallel performance
* Wrap C/C++ code

# Part 1: speed up your Python code

We want to integrate the function $f(x) = x^4 - 3x$.

In [ ]:
def f(x):
    y = x**4 - 3*x
    return y
    
def integrate_f(a, b, n):
    dx = (b - a) / n
    dx2 = dx / 2
    s = f(a) * dx2
    for i in range(1, n):
        s += f(a + i * dx) * dx
    s += f(b) * dx2
    return s

Now, let's time this:

In [ ]:
%timeit integrate_f(-100, 100, int(1e5))

Not too bad, but this can add up. Let's see if Cython can do better:

In [ ]:
%load_ext cython

In [ ]:
%%cython

def f2(x):
    y = x**4 - 3*x
    return y
    
def integrate_f2(a, b, n):
    dx = (b - a) / n
    dx2 = dx / 2
    s = f2(a) * dx2
    for i in range(1, n):
        s += f2(a + i * dx) * dx
    s += f2(b) * dx2
    return s

In [ ]:
%timeit integrate_f2(-100, 100, int(1e5))

That's a little bit faster, which is nice since all we did was to call Cython on the exact same code. But can we do better?

In [ ]:
%%cython

def f3(double x):
    y = x**4 - 3*x
    return y
    
def integrate_f3(double a, double b, int n):
    dx = (b - a) / n
    dx2 = dx / 2
    s = f3(a) * dx2
    for i in range(1, n):
        s += f3(a + i * dx) * dx
    s += f3(b) * dx2
    return s

In [ ]:
%timeit integrate_f3(-100, 100, int(1e5))

The final bit of "easy" Cython optimization is "declaring" the variables inside the function:

In [ ]:
%%cython

def f4(double x):
    y = x**4 - 3*x
    return y
    
def integrate_f4(double a, double b, int n):
    cdef:
        double dx = (b - a) / n
        double dx2 = dx / 2
        double s = f4(a) * dx2
        int i = 0
    for i in range(1, n):
        s += f4(a + i * dx) * dx
    s += f4(b) * dx2
    return s

In [ ]:
%timeit integrate_f4(-100, 100, int(1e5))

In [ ]:
%%cython

def f4(double x):
    y = x**4 - 3*x
    return y
    
def integrate_f4(double a, double b, int n):
    cdef double dx = (b - a) / n
    cdef double dx2 = dx / 2
    cdef double s = f4(a) * dx2
    cdef int i = 0
    for i in range(1, n):
        s += f4(a + i * dx) * dx
    s += f4(b) * dx2
    return s

4X speedup with so little effort is pretty nice. What else can we do?

Cython has a nice "-a" flag (for annotation) that can provide clues about why your code is slow.

In [ ]:
%%cython -a

def f4(double x):
    y = x**4 - 3*x
    return y
    
def integrate_f4(double a, double b, int n):
    cdef:
        double dx = (b - a) / n
        double dx2 = dx / 2
        double s = f4(a) * dx2
        int i = 0
    for i in range(1, n):
        s += f4(a + i * dx) * dx
    s += f4(b) * dx2
    return s

That's a lot of yellow still! How do we reduce this?

### Exercise: change the `f4` declaration to C

In [ ]:
%timeit integrate_f5(-100, 100, 10**5) 

In [ ]:
integrate_f5(-100, 100, 0)

# Part 2: work with NumPy arrays

The above is a very small subset of Python. Most scientific application deal not with single values, but with arrays of data.

In [ ]:
import numpy as np

def mean3filter(arr):
    arr_out = np.empty_like(arr)
    for i in range(1, arr.shape[0] - 1):
        arr_out[i] = np.sum(arr[i-1 : i+2]) / 3
    arr_out[0] = (arr[0] + arr[1]) / 2
    arr_out[-1] = (arr[-1] + arr[-2]) / 2
    return arr_out

In [ ]:
r = np.random.rand(10**5)

In [ ]:
%timeit mean3filter(r)

In [ ]:
%%cython
import cython
import numpy as np

@cython.boundscheck(False)
def mean3filter2(double[::1] arr):
    cdef double[::1] arr_out = np.empty_like(arr)
    cdef int i
    for i in range(1, arr.shape[0]-1):
        arr_out[i] = np.sum(arr[i-1 : i+2]) / 3
    arr_out[0] = (arr[0] + arr[1]) / 2
    arr_out[-1] = (arr[-1] + arr[-2]) / 2
    return np.asarray(arr_out)

In [ ]:
%timeit mean3filter2(r)

Rubbish! How do we fix this?

### Exercise: use `%%cython -a` to speed up the code

In [ ]:
arr_out = np.zeros_like(r)
%timeit mean3filter3(r, arr_out)

# Part 3: write parallel code

**Warning:**: Dragons afoot.

In [ ]:
%%cython -a
import cython
from cython.parallel import prange
import numpy as np

@cython.boundscheck(False)
def mean3filter4(double[::1] arr, double[::1] out):
    cdef int i, j, k = arr.shape[0]-1
    with nogil:
        for i in prange(1, k-1, schedule='static',
                        chunksize=(k-2) // 2, num_threads=2):
            for j in range(i-1, i+1):
                out[i] += arr[j]
            out[i] /= 3
    out[0] = (arr[0] + arr[1]) / 2
    out[-1] = (arr[-1] + arr[-2]) / 2
    return np.asarray(out) 

In [ ]:
rin = np.random.rand(10**8)
rout = np.empty_like(rin)

%timeit mean3filter4(rin, rout)

In [ ]:
# comparing to single-threaded...
rin = np.random.rand(10**8)
rout = np.empty_like(rin)

%timeit mean3filter3(rin, rout)

# Part 4: interact with C/C++ code

In [ ]:
%%cython -a
# distutils: language=c++
import cython
from libcpp.vector cimport vector


@cython.boundscheck(False)
def build_list_with_vector(double[::1] in_arr):
    cdef vector[double] out
    cdef int i
    for i in range(in_arr.shape[0]):
        out.push_back(in_arr[i])
    return out

In [ ]:
build_list_with_vector(np.random.rand(10))

## Example: C++ int graph

In [ ]:
%%cython -a
#distutils: language=c++
from cython.operator cimport dereference as deref, preincrement as inc

from libcpp.vector cimport vector
from libcpp.map cimport map as cppmap

cdef class Graph:
    cdef cppmap[int, vector[int]] _adj
    
    cpdef int has_node(self, int node):
        return self._adj.find(node) != self._adj.end()
    
    cdef void add_node(self, int new_node):
        cdef vector[int] out
        if not self.has_node(new_node):
            self._adj[new_node] = out
            
    def add_edge(self, int u, int v):
        self.add_node(u)
        self.add_node(v)
        self._adj[u].push_back(v)
        self._adj[v].push_back(u)
        
    def __getitem__(self, int u):
        return self._adj[u]
    
    cdef vector[int] _degrees(self):
        cdef vector[int] deg
        cdef int first = 0
        cdef vector[int] edges
        cdef cppmap[int, vector[int]].iterator it = self._adj.begin()
        while it != self._adj.end():
            deg.push_back(deref(it).second.size())
            it = inc(it)
        return deg
            
    def degrees(self):
        return self._degrees()
        

In [ ]:
g0 = Graph() 

In [ ]:
g0.add_edge(1, 5)
g0.add_edge(1, 6)

In [ ]:
g0[1]

In [ ]:
g0.has_node(1)

In [ ]:
g0.degrees()

In [ ]:
import networkx as nx
g = nx.barabasi_albert_graph(100000, 6)
with open('graph.txt', 'w') as fout:
    for u, v in g.edges_iter():
        fout.write('%i,%i\n' % (u, v))

In [ ]:
%timeit list(g.degree())

In [ ]:
myg = Graph()
def line2edges(line):
    u, v = map(int, line.rstrip().split(','))
    return u, v

edges = map(line2edges, open('graph.txt'))

for u, v in edges:
    myg.add_edge(u, v)

In [ ]:
%timeit mydeg = myg.degrees()

# Using Cython in production code

Use `setup.py` to build your Cython files.

```python
from distutils.core import setup
from distutils.extension import Extension
from Cython.Distutils import build_ext

import numpy as np

setup(
  cmdclass = {'build_ext': build_ext},
  ext_modules = [
    Extension("prange_demo", ["prange_demo.pyx"],
              include_dirs=[np.get_include()],
              extra_compile_args=['-fopenmp'],
              extra_link_args=['-fopenmp', '-lgomp']),
  ]
)
```

### Exercise

Write a Cython module with a setup.py to run the mean-3 filter, then import from the notebook.

- Write a mean3.pyx
- Write a corresponding setup.py
- Run python setup.py build_ext -i

NOTE: Don't add extra compiler or link args!

In [ ]:
from mean3 import mean3filter3
rin = np.random.rand(10)
rout = np.zeros(10)
mean3filter3(rin, rout)
print(rout) 

In [ ]:
mean3filter3?

# Numba and interacting with SciPy

In [ ]:
import numpy as np


def addarr(x, y):
    result = np.zeros_like(x)
    for i in range(x.size):
        result[i] = x[i] + y[i]
    return result

How long does this take in pure Python?

In [ ]:
n = int(1e6)
a = np.random.rand(n)
b = np.random.rand(n)

In [ ]:
%timeit -r 1 -n 1 addarr(a, b)

About half a second on my machine. Let's try with Numba using its JIT decorator:

In [ ]:
import numba

addarr_nb = numba.jit(addarr) 

In [ ]:
%timeit -r 1 -n 1 addarr_nb(a, b)

The first time it runs, it's only a tiny bit faster. That's because of the nature of JITs: they only compile code *as it is being run*, in order to use object type information of the objects passed into the function. (Note that, in Python, the arguments `a` and `b` to `addarr` could be anything: an array, as expected, but also a list, a tuple, even a `Banana`, if you've defined such a class, and the meaning of the function body is different for each of those types.)

Let's see what happens the next time we run it:

In [ ]:
%timeit -r 1 -n 1 addarr_nb(a, b)

Whoa! Now the code takes 5ms, about 100 times faster than the pure Python version. And the NumPy equivalent?

In [ ]:
%timeit -r 1 -n 1 a + b

Only marginally faster than Numba, even though NumPy addition is implemented in highly optimised C code. And, for some data types, Numba even beats NumPy:

In [ ]:
r = np.random.randint(0, 128, size=n).astype(np.uint8)
s = np.random.randint(0, 128, size=n).astype(np.uint8)

In [ ]:
%timeit -r 1 -n 1 r + s

In [ ]:
%timeit -r 1 -n 1 addarr_nb(r, s)

In [ ]:
%timeit -r 1 -n 1 addarr_nb(r, s)

In [ ]:
def add_numpy(a, b):
    return a + b

In [ ]:
add_numpy_jit = numba.jit(add_numpy)

In [ ]:
%timeit -r 1 -n 1 add_numpy_jit(r, s)

### Exercise

Write a mean filter using Numba, and time it, using `-r 1 -n 1` as above. How does it compare to Cython?

In [ ]:
import numpy as np

# decorate here...
def mean3filter_nb(arr, arr_out):
    pass  # ... and fill this in

In [ ]:
rin = np.random.rand(10**8)
rout = np.zeros_like(rin)

%timeit -r 1 -n 1 mean3filter_nb(rin, rout)  # jit warmup

In [ ]:
rin = np.random.rand(10**8)
rout = np.zeros_like(rin)

%timeit -r 1 -n 1 mean3filter_nb(rin, rout)  # faster!

## SciPy LowLevelCallable

See:
- https://ilovesymposia.com/2017/03/12/scipys-new-lowlevelcallable-is-a-game-changer/
- https://ilovesymposia.com/2017/03/15/prettier-lowlevelcallables-with-numba-jit-and-decorators/

In [ ]:
import numba
from numba import cfunc, carray
from numba.types import intc, CPointer, float64, intp, voidptr
from scipy import LowLevelCallable

def jit_filter_function(filter_function):
    jitted_function = numba.jit(filter_function, nopython=True)
    
    @cfunc(intc(CPointer(float64), intp, CPointer(float64), voidptr))
    def wrapped(values_ptr, len_values, result, data):
        values = carray(values_ptr, (len_values,), dtype=float64)
        result[0] = jitted_function(values)
        return 1
    
    return LowLevelCallable(wrapped.ctypes)

In [ ]:
@jit_filter_function
def fmin(values):
    result = np.inf
    for v in values:
        if v < result:
            result = v
    return result

In [ ]:
image = np.random.random((2048, 2048))
footprint = np.ones((3, 3))
%timeit ndi.generic_filter(image, fmin, footprint=footprint)

Cython can also be used to create LowLevelCallables:

In [ ]:
%%cython --name=test9

from libc.stdint cimport intptr_t
from numpy.math cimport INFINITY

cdef api int erosion_kernel(double* input_arr_1d, intptr_t filter_size,
                            double* return_value, void* user_data):
    cdef:
        double[:] input_arr
        ssize_t i
        double result = INFINITY
    for i in range(filter_size):
        if input_arr_1d[i] < return_value[0]:
            result = input_arr_1d[i]
    return_value[0] = result
    return 1

In [ ]:
import test9

In [ ]:
from scipy import LowLevelCallable, ndimage
import sys

def erosion_fast(image, footprint):
    out = ndimage.generic_filter(
            image,
            LowLevelCallable.from_cython(test9, name='erosion_kernel'),
            footprint=footprint
    )
    return out

In [ ]:
%timeit erosion_fast(image, footprint)

### Exercise

Write a mean filter using `generic_filter`, and time it. How does it compare to the above?

In [ ]:
# write a filter function here...

def mean3filter_ndi(arr_in, arr_out):
    pass  # ... then fill this in

In [ ]:
rin = np.random.rand(10**8)
rout = np.zeros_like(rin)

%timeit -r 1 -n 1 mean3filter_ndi(rin, rout)

In [ ]:
rin = np.random.rand(10**8)
rout = np.zeros_like(rin)

%timeit -r 1 -n 1 mean3filter3(rin, rout)

This is about 2-3x slower than Cython and 6x slower than Numba, probably accounting for some overhead of allowing generic filters. (But to be honest I don't know why this is so much slower!)

## Concluding remarks

In [ ]:
rout.shape

Some pros and cons about Cython and Numba:

- Cython pros:
  * very wide support
  * easy to distribute compiled code to most users
  * quite developed optimizing workflow (i.e. cython -a)
- Cython cons:
  * need to use a new language


- Numba pros:
  * quite easy to use, especially if you're starting from Cython code
  * often eye-popping, face-melting performance
- Numba cons:
  * hard to install outside of conda
  * hard to optimise. If it's slow, you have to guess (though they are helpful on mailing list)
  * many parts of Python still unsupported, e.g. dicts.
  * project still young and some people are paranoid that it could disappear